In [1]:
from ipyleaflet import Map, Heatmap, Marker, GeoData, LayersControl, LayerGroup, Icon, WidgetControl
from ipywidgets import HTML
import base64
import numpy as np
import geopandas as gpd
import pandas as pd
import base64
from ipyleaflet import Map, GeoData, Marker, Icon, LayerGroup, WidgetControl, LayersControl, basemaps
from ipywidgets import HTML, Button, Dropdown, VBox, HBox, Label
import warnings
warnings.filterwarnings('ignore')

In [4]:
def crear_leyenda_rutas_completa(titulo, columna_valor, umbral, alta_count, baja_count, min_rutas, max_rutas, incluye_heatmap):
    """Crea una leyenda HTML completa para el mapa"""

    contenido = f"""
    <div style="
        background: white;
        padding: 10px;
        border-radius: 5px;
        border: 1px solid #ccc;
        font-family: Arial;
        font-size: 12px;
        max-width: 280px;
    ">

        <div style="margin-bottom: 8px;">
            <strong>Percepción de Accesibilidad</strong><br>
            <div style="display: flex; align-items: center; margin: 2px 0;">
                <div style="width: 12px; height: 12px; background: #2ecc71; margin-right: 5px; border-radius: 50%;"></div>
                Alta (> {umbral}): {alta_count} puntos
            </div>
            <div style="display: flex; align-items: center; margin: 2px 0;">
                <div style="width: 12px; height: 12px; background: #e74c3c; margin-right: 5px; border-radius: 50%;"></div>
                Baja (≤ {umbral}): {baja_count} puntos
            </div>
        </div>

        <div style="margin-bottom: 8px;">
            <strong>Rutas de Transporte</strong><br>
            <div style="display: flex; flex-wrap: wrap; gap: 8px; margin-top: 5px;">
                <div style="display: flex; flex-direction: column; align-items: center;">
                    <div style="width: 20px; height: 12px; background: #f0f0f0; border: 1px solid #999; margin-bottom: 2px;"></div>
                    <div style="font-size: 10px; text-align: center;">0</div>
                </div>
                <div style="display: flex; flex-direction: column; align-items: center;">
                    <div style="width: 20px; height: 12px; background: #fee5d9; border: 1px solid #999; margin-bottom: 2px;"></div>
                    <div style="font-size: 10px; text-align: center;">1-5</div>
                </div>
                <div style="display: flex; flex-direction: column; align-items: center;">
                    <div style="width: 20px; height: 12px; background: #fcae91; border: 1px solid #999; margin-bottom: 2px;"></div>
                    <div style="font-size: 10px; text-align: center;">6-10</div>
                </div>
                <div style="display: flex; flex-direction: column; align-items: center;">
                    <div style="width: 20px; height: 12px; background: #fb6a4a; border: 1px solid #999; margin-bottom: 2px;"></div>
                    <div style="font-size: 10px; text-align: center;">11-15</div>
                </div>
                <div style="display: flex; flex-direction: column; align-items: center;">
                    <div style="width: 20px; height: 12px; background: #cb181d; border: 1px solid #999; margin-bottom: 2px;"></div>
                    <div style="font-size: 10px; text-align: center;">16-56</div>
                </div>
            </div>
        </div>
    """

    if incluye_heatmap:
        contenido += """
        <div style="margin-bottom: 5px;">
            <strong>Heatmap Percepción</strong><br>
            <div style="display: flex; align-items: center; margin: 2px 0;">
                <div style="width: 40px; height: 8px; background: linear-gradient(to right, blue, cyan, lime, yellow, red); margin-right: 5px;"></div>
                Baja → Alta
            </div>
        </div>
        """

    return HTML(contenido)

def crear_controles_hamburguesa_rapido(markers_data, columnas_filtro, nombres_filtros, callback_filtros):
    """Versión ultra rápida usando la propiedad visible de Marker"""

    # Obtener valores únicos del primer marcador (todos tienen los mismos datos)
    df_unique = pd.DataFrame([{col: data[col] for col in columnas_filtro} for data in markers_data])

    # Crear dropdowns con nombres personalizados
    dropdowns = {}
    for i, columna in enumerate(columnas_filtro):
        valores_unicos = sorted(list(df_unique[columna].unique()))
        nombre_personalizado = nombres_filtros[i]  # Usar el nombre personalizado

        dropdown = Dropdown(
            options=['Todos'] + valores_unicos,
            value='Todos',
            description=f'{nombre_personalizado}:',
            layout=dict(width='280px', height='28px'),
            style={'description_width': '120px'}
        )
        dropdowns[columna] = dropdown

    # Función de filtrado ULTRA RÁPIDA usando la propiedad visible
    def filtrar_marcadores(change):
        valores_seleccionados = {}
        for columna in columnas_filtro:
            valor = dropdowns[columna].value
            if valor != 'Todos':
                valores_seleccionados[columna] = valor

        # Llamar al callback con los filtros activos
        callback_filtros(valores_seleccionados)

    for dropdown in dropdowns.values():
        dropdown.observe(filtrar_marcadores, names='value')

    # Botón de reinicio con mejor diseño
    reset_btn = Button(
        description="↻",
        button_style='',
        layout=dict(width='35px', height='28px'),
        tooltip="Resetear filtros",
        style={'font_weight': 'bold', 'font_size': '14px'}
    )

    def reset_filtros(btn):
        for dropdown in dropdowns.values():
            dropdown.value = 'Todos'
        # Reset instantáneo: hacer todos visibles
        callback_filtros({})

    reset_btn.on_click(reset_filtros)

    # Botón de menú hamburguesa
    menu_btn = Button(
        description="☰",
        button_style='',
        layout=dict(width='35px', height='28px'),
        tooltip="Mostrar/ocultar filtros"
    )

    # Título "Filtros" separado
    titulo_filtros = Label(
        value="Filtros",
        layout=dict(width='90px', height='28px', margin='0 0 0 10px')
    )

    # Contenedor de filtros
    filtros_container = VBox(
        [dropdowns[col] for col in columnas_filtro],
        layout=dict(
            padding='5px',
            margin='2px'
        )
    )

    # Estado inicial - filtros visibles
    filtros_visibles = True

    # Función para toggle del menú
    def toggle_menu(btn):
        nonlocal filtros_visibles
        filtros_visibles = not filtros_visibles
        if filtros_visibles:
            filtros_container.layout.display = 'flex'
            menu_btn.description = "☰"
        else:
            filtros_container.layout.display = 'none'
            menu_btn.description = "☰"

    menu_btn.on_click(toggle_menu)

    # Header con controles mejorado
    header = HBox([
        menu_btn,
        titulo_filtros,
        reset_btn
    ], layout=dict(
        padding='5px 10px',
        justify_content='space-between',
        width='320px',
        align_items='center'
    ))

    return VBox([
        header,
        filtros_container
    ], layout=dict(
        padding='0px',
        border='1px solid #ddd',
        background='white',
        width='320px'
    ))


def crear_mapa_capa_agrupada(
    df_puntos,
    gdf_poligonos,
    columna_valor,
    columnas_filtro,
    nombres_filtros,
    columna_rutas='NO_RUTAS',
    umbral=0,
    incluir_heatmap=True
):
    """
    Versión con UNA capa agrupada para todas las rutas.
    Incluye un heatmap basado en pers_param_modelo_a,
    con normalización automática al rango [0,1] según valores reales.
    LOS FILTROS AHORA SE APLICAN TANTO A MARCADORES COMO AL HEATMAP
    """

    # 1. Crear mapa base
    mapa = Map(
        center=(df_puntos['latitude'].mean(), df_puntos['longitude'].mean()),
        zoom=15,
        basemap=basemaps.OpenStreetMap.Mapnik,
        layout=dict(width='100%', height='600px')
    )

    # 2. Capa agrupada para polígonos de rutas
    capa_rutas_agrupada = LayerGroup(name="Rutas de Transporte")
    mapa.add_layer(capa_rutas_agrupada)

    min_rutas = gdf_poligonos[columna_rutas].min()
    max_rutas = gdf_poligonos[columna_rutas].max()

    categorias_rutas = [
        {'min': 0, 'max': 0, 'color': '#f0f0f0', 'nombre': '0 rutas'},
        {'min': 1, 'max': 5, 'color': '#fee5d9', 'nombre': '1-5 rutas'},
        {'min': 6, 'max': 10, 'color': '#fcae91', 'nombre': '6-10 rutas'},
        {'min': 11, 'max': 15, 'color': '#fb6a4a', 'nombre': '11-15 rutas'},
        {'min': 16, 'max': 56, 'color': '#cb181d', 'nombre': '16-56 rutas'}
    ]

    for categoria in categorias_rutas:
        if categoria['min'] == 0 and categoria['max'] == 0:
            gdf_filtrado = gdf_poligonos[gdf_poligonos[columna_rutas] == 0]
        else:
            gdf_filtrado = gdf_poligonos[
                (gdf_poligonos[columna_rutas] >= categoria['min']) &
                (gdf_poligonos[columna_rutas] <= categoria['max'])
            ]

        if len(gdf_filtrado) > 0:
            capa_categoria = GeoData(
                geo_dataframe=gdf_filtrado,
                style={
                    'color': 'gray',
                    'weight': 1,
                    'fillColor': categoria['color'],
                    'fillOpacity': 0.7,
                    'opacity': 0.6
                },
                name=f"Rutas {categoria['nombre']}",
                hover_style={
                    'fillColor': 'yellow',
                    'fillOpacity': 0.8,
                    'weight': 2
                }
            )
            capa_rutas_agrupada.add_layer(capa_categoria)

    # 3. FUNCIÓN PARA CREAR HEATMAP (reutilizable)
    def crear_heatmap(df_filtrado):
        if not incluir_heatmap or 'pers_param_modelo_a' not in df_filtrado.columns:
            return None

        # Rango real (ignorando NaN)
        min_val = df_filtrado['pers_param_modelo_a'].min(skipna=True)
        max_val = df_filtrado['pers_param_modelo_a'].max(skipna=True)

        if pd.isna(min_val) or pd.isna(max_val):
            return None

        # Dividir rango en negativo (min_neg .. 0) y positivo (0 .. max_pos)
        min_neg = min(min_val, 0.0)
        max_pos = max(max_val, 0.0)

        def map_to_0_1_center0(v, min_neg=min_neg, max_pos=max_pos):
            """
            Mapea v en [-inf, inf] a [0,1] asegurando que 0 -> 0.5.
            - si v <= 0: mapea linealmente desde min_neg -> 0  a 0.0 -> 0.5
            - si v > 0:  mapea linealmente desde 0 -> max_pos  a 0.5 -> 1.0
            Si min_neg == 0 (no negativos) o max_pos == 0 (no positivos) maneja ese caso.
            """
            if v <= 0:
                # si no hay negativos, colocar todo por debajo de 0 en 0.25 (opcional) o 0.0
                if min_neg == 0:
                    return 0.0  # todos >=0 en esta rama (raro)
                else:
                    return 0.5 * ((v - min_neg) / (0 - min_neg))
            else:
                if max_pos == 0:
                    return 1.0  # todos <=0 en esta rama (raro)
                else:
                    return 0.5 + 0.5 * (v / max_pos)

        # Aplicar mapeo seguro y evitar NaN
        df_filtrado['valor_norm'] = df_filtrado['pers_param_modelo_a'].apply(
            lambda x: map_to_0_1_center0(x) if not pd.isna(x) else 0.5
        )

        # Construir la lista (lat, lon, intensidad)
        heatmap_data = [
            (float(row['latitude']), float(row['longitude']), float(row['valor_norm']))
            for _, row in df_filtrado.iterrows()
            if not np.isnan(row['latitude']) and not np.isnan(row['longitude'])
        ]

        # Gradiente divergente: azul (baja) -> blanco (0) -> rojo (alta)
        heatmap_layer = Heatmap(
            locations=heatmap_data,
            radius=25,        # AUMENTADO: puntos de calor más grandes
            blur=8,           # DISMINUIDO: bordes menos difuminados (más sólidos)
            max_zoom=18,
            gradient={
                0.0: 'darkblue',   # Colores más saturados
                0.2: 'blue',
                0.4: 'cyan',
                0.5: 'lime',      
                0.6: 'yellow',
                0.8: 'orange',
                1.0: 'darkred'     # Rojo más intenso
            },
            opacity= 1.0,
            name="Heatmap - Percepción"
        )
        return heatmap_layer

    # 4. Crear heatmap inicial
    heatmap_actual = crear_heatmap(df_puntos)
    if heatmap_actual:
        mapa.add_layer(heatmap_actual)

    # 5. CAPA DE MARCADORES DE ENCUESTA
    all_markers_group = LayerGroup(name="Puntos de Encuesta")
    mapa.add_layer(all_markers_group)

    markers_data = []

    for i, row in df_puntos.iterrows():
        lat = float(row['latitude'])
        lon = float(row['longitude'])
        val = float(row[columna_valor])

        color = '#2ecc71' if val > umbral else '#e74c3c'
        categoria = 'Alta' if val > umbral else 'Baja'

        filtros_texto = "<br>".join([f"<b>{col}:</b> {row[col]}" for col in columnas_filtro])

        pers_param_text = ""
        if 'pers_param_modelo_a' in df_puntos.columns:
            pers_param_val = float(row['pers_param_modelo_a'])
            pers_param_text = f"<p><b>pers_param_modelo_a:</b> {pers_param_val:.2f}</p>"

        popup_text = f"""
        <div style="font-family: Arial; width: 250px;">
            <b>Ubicación {i+1}</b><br><hr>
            <p><b>{columna_valor}:</b> {val:.2f} (<span style="color:{color}">{categoria}</span>)</p>
            {pers_param_text}
            {filtros_texto}
            <p><b>Coords:</b> {lat:.4f}, {lon:.4f}</p>
        </div>
        """

        popup = HTML(popup_text)

        svg_icon = f"""
        <svg width="20" height="20" xmlns="http://www.w3.org/2000/svg">
            <circle cx="10" cy="10" r="8" fill="{color}" stroke="white" stroke-width="2"
                    fill-opacity="0.9" stroke-opacity="1"/>
        </svg>
        """

        svg_encoded = base64.b64encode(svg_icon.encode('utf-8')).decode('utf-8')
        icon_url = f"data:image/svg+xml;base64,{svg_encoded}"

        icon = Icon(icon_url=icon_url, icon_size=[20, 20], icon_anchor=[10, 10])

        marker = Marker(
            location=(lat, lon),
            draggable=False,
            title=f"{columna_valor}: {val:.2f} - {categoria}",
            popup=popup,
            icon=icon
        )

        all_markers_group.add_layer(marker)

        marker_info = {'marker': marker, 'valor': val, 'categoria': categoria}
        for col in columnas_filtro:
            marker_info[col] = row[col]
        markers_data.append(marker_info)

    # 6. FUNCIÓN PRINCIPAL DE FILTRADO (actualiza marcadores Y heatmap)
    def aplicar_filtros(filtros_activos):
        # Filtrar marcadores
        for marker_info in markers_data:
            visible = True
            for col, valor_esperado in filtros_activos.items():
                if marker_info[col] != valor_esperado:
                    visible = False
                    break
            marker_info['marker'].visible = visible

        # Filtrar datos para heatmap
        df_filtrado = df_puntos.copy()
        for col, valor in filtros_activos.items():
            df_filtrado = df_filtrado[df_filtrado[col] == valor]

        # Actualizar heatmap
        if incluir_heatmap:
            # Remover heatmap anterior si existe
            for layer in mapa.layers:
                if isinstance(layer, Heatmap):
                    mapa.remove_layer(layer)
                    break

            # Crear nuevo heatmap con datos filtrados
            nuevo_heatmap = crear_heatmap(df_filtrado)
            if nuevo_heatmap:
                mapa.add_layer(nuevo_heatmap)

        actualizar_leyenda()

    # 7. LEYENDA y controles
    def contar_marcadores_visibles():
        alta_count = sum(1 for d in markers_data if d['marker'].visible and d['categoria'] == 'Alta')
        baja_count = sum(1 for d in markers_data if d['marker'].visible and d['categoria'] == 'Baja')
        return alta_count, baja_count

    def actualizar_leyenda():
        nonlocal leyenda_control
        alta_count, baja_count = contar_marcadores_visibles()

        # Verificar si hay heatmap visible
        heatmap_visible = any(isinstance(layer, Heatmap) for layer in mapa.layers)

        nueva_leyenda = crear_leyenda_rutas_completa(
            "Percepción vs Rutas de Transporte",
            columna_valor,
            umbral,
            alta_count,
            baja_count,
            min_rutas,
            max_rutas,
            incluir_heatmap and heatmap_visible
        )
        mapa.remove_control(leyenda_control)
        leyenda_control = WidgetControl(widget=nueva_leyenda, position='bottomleft')
        mapa.add_control(leyenda_control)

    alta_count, baja_count = contar_marcadores_visibles()
    heatmap_visible = any(isinstance(layer, Heatmap) for layer in mapa.layers)
    leyenda = crear_leyenda_rutas_completa(
        "Percepción vs Rutas de Transporte",
        columna_valor,
        umbral,
        alta_count,
        baja_count,
        min_rutas,
        max_rutas,
        incluir_heatmap and heatmap_visible
    )
    leyenda_control = WidgetControl(widget=leyenda, position='bottomleft')
    mapa.add_control(leyenda_control)

    # 8. CONTROLES con la nueva función de filtrado
    controles = crear_controles_hamburguesa_rapido(
        markers_data, columnas_filtro, nombres_filtros, aplicar_filtros
    )
    control_widget = WidgetControl(widget=controles, position='topright')
    mapa.add_control(control_widget)

    # 9. CREAR O REEMPLAZAR EL LAYERSCONTROL (evita duplicados)
    for ctrl in list(mapa.controls):
        if isinstance(ctrl, LayersControl):
            mapa.remove_control(ctrl)

    layers_control = LayersControl(position='topright')
    mapa.add_control(layers_control)


    return mapa


In [5]:
gdf_manzanas = gpd.read_file('data/ACCESIBILIDAD_RUTAS_TRANSPORTE.shp')
df = pd.read_csv("data/percepcion_ofuscada.csv")

mapa_final = crear_mapa_capa_agrupada(
    df_puntos=df,
    gdf_poligonos=gdf_manzanas,
    columna_valor='pers_param_modelo_a',
    columnas_filtro=['menor_igualsecu', 'edad_grupo60', 'edad_mediana','acu_desacu_tp_unica2','acu_desacu_prefiero_vp2','acu_desacu_act_diarias2','acu_desacu_vivir2','acu_desacu_act_prefiero2', 'acu_desacu_satis2'],
    nombres_filtros=['secundaria o menos','edad 60', 'edad 42', 'tp única', 'prefiero', 'act_diarias', 'vivir', 'act_pref', 'act_satis'],
    columna_rutas='NO_RUTAS',
    umbral=0,
    incluir_heatmap=True
)

display(mapa_final)

Map(center=[np.float64(19.71922162806662), np.float64(-101.22390326773115)], controls=(ZoomControl(options=['p…